In [ ]:
from flask import Flask, render_template, request, redirect, url_for, flash
import sqlite3
from werkzeug.security import generate_password_hash
from datetime import datetime
import re

app = Flask(__name__)

app.secret_key = '2de4621b6cfc0f04c7dec6c7a46e0299'

# Initialize the SQLite database
def init_db():
    conn = sqlite3.connect('users.db')
    c = conn.cursor()
    # Create users table if it doesn't exist
    c.execute('''CREATE TABLE IF NOT EXISTS users
                 (id INTEGER PRIMARY KEY AUTOINCREMENT,
                  first_name TEXT NOT NULL,
                  last_name TEXT NOT NULL,
                  email TEXT NOT NULL UNIQUE,
                  password TEXT NOT NULL,
                  password_hint TEXT NOT NULL,
                  created_at TEXT NOT NULL)''')
    conn.commit()
    conn.close()

@app.route('/')
def home():
    return '''
    <h1> Patient Database </h1>
    <a href="register">Go to registration</a><br>
    <a href="users">View users</a><br>
    '''

# Email validation using regex
def is_valid_email(email):
    email_regex = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
    return re.match(email_regex, email)

# Password strength validation
def is_password_strong(password):
    if len(password) < 8:
        return False
    if not re.search(r'[A-Z]', password):
        return False
    if not re.search(r'[a-z]', password):
        return False
    if not re.search(r'[0-9]', password):
        return False
    return True

# Route to display the registration form
@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        # Retrieve form data
        first_name = request.form['first_name']
        last_name = request.form['last_name']
        email = request.form['email']
        password = request.form['password']
        password_hint = request.form['password_hint']

        print(f"Received: {first_name}, {last_name}, {email}, {password_hint}")

        # Hash the password
        # hashed_password = generate_password_hash(password, method='sha256')
        hashed_password = generate_password_hash(password, method='pbkdf2:sha256')

        # Validate form data
        if not first_name or not last_name or not email or not password:
            flash('All fields are required.')
            print('All fields are required.')
            return render_template('register.html')

        

        # Validate first and last names contain only letters
        if not re.match("^[A-Za-z]+$", first_name):
            flash('First name should contain only letters.')
            print("First name should contain only letters.")
            return render_template('register.html')
        if not re.match("^[A-Za-z]+$", last_name):
            flash('Last name should contain only letters.')
            print("Last name should contain only letters.")
            return render_template('register.html')

        # Validate email format
        if not is_valid_email(email):
            flash('Invalid email format. Please enter a valid email.')
            print('Invalid email format. Please enter a valid email.')
            return render_template('register.html')

        # Validate password strength
        if not is_password_strong(password):
            flash('Password must be at least 8 characters long, include at least one uppercase letter, one lowercase letter, and one number.')

            return render_template('register.html')

        # Get the current timestamp
        created_at = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        # Insert user data into the database
        try:
            print("Attempting to add to db")
            conn = sqlite3.connect('users.db')
            c = conn.cursor()
            c.execute("INSERT INTO users (first_name, last_name, email, password, password_hint, created_at) VALUES (?, ?, ?, ?, ?, ?)", 
                      (first_name, last_name, email, hashed_password, password_hint, created_at,))      
            conn.commit()
            conn.close()

            print("User added successfully.")

            # Redirect to the success page
            return redirect(url_for('success', first_name=first_name))
        except sqlite3.IntegrityError:
            flash('Email already registered. Please use a different email.')
            return render_template('register.html')

    return render_template('register.html')

# Route to display the success page
@app.route('/success/<first_name>')
def success(first_name):
    return f"User {first_name} registered successfully!\n<a href=\"/\">Go to homepage</a>"

# Route to list all users (excluding passwords)
@app.route('/users')
def list_users():
    conn = sqlite3.connect('users.db')
    c = conn.cursor()
    c.execute('SELECT id, first_name, last_name, email, created_at FROM users')
    users = c.fetchall()
    conn.close()
    print(users)
    return render_template('users.html', users=users)

# Route to delete a user by ID
@app.route('/delete/<int:user_id>')
def delete_user(user_id):
    conn = sqlite3.connect('users.db')
    c = conn.cursor()
    c.execute('DELETE FROM users WHERE id = ?', (user_id,))
    conn.commit()
    conn.close()
    return redirect(url_for('list_users'))


# Initialize the database and run the Flask app
if __name__ == '__main__':
    init_db()
    app.run(port=5501, debug=False)  # Disable debug mode

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5501
Press CTRL+C to quit
127.0.0.1 - - [21/Nov/2024 15:47:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2024 15:47:01] "GET /register HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2024 15:47:13] "POST /register HTTP/1.1" 302 -
127.0.0.1 - - [21/Nov/2024 15:47:13] "GET /success/b HTTP/1.1" 200 -


Received: b, b, a@b.com, 1
Attempting to add to db
User added successfully.


127.0.0.1 - - [21/Nov/2024 15:47:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2024 15:47:16] "GET /users HTTP/1.1" 200 -


[(1, 'b', 'b', 'a@b.com', '2024-11-21 15:47:13')]


...

Received: b, b, b@c.com, b


127.0.0.1 - - [21/Nov/2024 15:47:38] "POST /register HTTP/1.1" 302 -
127.0.0.1 - - [21/Nov/2024 15:47:38] "GET /success/b HTTP/1.1" 200 -


Received: b, b, b@c.com, 1
Attempting to add to db
User added successfully.


127.0.0.1 - - [21/Nov/2024 15:47:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2024 15:47:41] "GET /users HTTP/1.1" 200 -


[(1, 'b', 'b', 'a@b.com', '2024-11-21 15:47:13'), (2, 'b', 'b', 'b@c.com', '2024-11-21 15:47:38')]
